### Import các thư viện

In [1]:
import pypyodbc
import pandas as pd
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from scipy.spatial.distance import cdist

from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist, jaccard
from sqlalchemy import create_engine
import urllib

Ta chỉ cần tập trung vào 3 cột nhất định để xây dựng thuật toán đề xuất là mã định danh bộ phim và người xem tương ứng với số điểm rating (stars).

In [3]:
movie_id = "imdb_id"
author_username = "author"
ratings = "stars"

In [4]:
train_df = pd.read_csv("training_set.csv")
test_df = pd.read_csv("testing_set.csv")

@@Todo

Sử dụng pivot table để tạo ma trận user-item giữa user và các bộ phim. Giá trị của ma trận được thể hiện thông qua giá trị đánh giá (rating).

In [5]:
df_matrix = train_df.pivot_table(index=author_username, columns=movie_id, values=ratings)

df_matrix.reset_index(inplace=True)

In [6]:
df_matrix

imdb_id,author,tt0000417,tt0010323,tt0013442,tt0015648,tt0015864,tt0017136,tt0020629,tt0021749,tt0021814,...,tt9784708,tt9784798,tt9794630,tt9806370,tt9831136,tt9844522,tt9845564,tt9848626,tt9866072,tt9907782
0,007Waffles,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,00Yasser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,04GreatFlick,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0maro0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0w0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12532,zzapper-2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12533,zzhhhqing,NaN,NaN,NaN,NaN,10.0,9.0,NaN,10.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12534,zzyzx000-124-271558,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12535,~AleXa~,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Chuẩn hóa ma trận

In [6]:
cols = df_matrix.columns.to_list()[1:]
matrix_norm = df_matrix[cols].subtract(df_matrix[cols].mean(axis=1), axis = 'rows')
matrix_norm.replace(np.nan,0,inplace=True)

In [7]:
matrix_norm

imdb_id,tt0000417,tt0010323,tt0013442,tt0015648,tt0015864,tt0017136,tt0020629,tt0021749,tt0021814,tt0021884,...,tt9784708,tt9784798,tt9794630,tt9806370,tt9831136,tt9844522,tt9845564,tt9848626,tt9866072,tt9907782
0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12532,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12533,0.0,0.0,0.0,0.0,1.117021,0.117021,0.0,1.117021,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12534,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12535,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


@@Todo

Collaborative Filtering là gì? Collaborative Filtering là một kỹ thuật có thể lọc ra các mục mà người dùng có thể thích trên cơ sở phản ứng của những người dùng tương tự. Nó hoạt động bằng cách tìm kiếm một nhóm lớn người và tìm một nhóm người dùng nhỏ hơn có sở thích tương tự như một người dùng cụ thể.

Trong khai thác dữ liệu, độ tương tự (similarity measure) đề cập đến khoảng cách với các kích thước biểu thị các tính năng của đối tượng dữ liệu, trong tập dữ liệu. Nếu khoảng cách này càng nhỏ thì mức độ tương đồng càng cao, còn khi khoảng cách này càng lớn thì mức độ tương đồng càng thấp.

Một số biện pháp tương tự phổ biến là: 

1. Khoảng cách Euclide.
2. Khoảng cách Manhattan.
3. Tương tự Jaccard.
4. Khoảng cách Minkowski.
5. Độ đồng dạng Cosine.

Độ tương tự cosine là một số liệu, hữu ích trong việc xác định mức độ giống nhau của các đối tượng dữ liệu bất kể kích thước của chúng. Chúng ta có thể đo lường sự giống nhau giữa hai câu trong Python bằng **Cosine Similarity**.

In [ ]:
df_matrix = df_matrix.copy()

In [10]:
# Sử dụng phương pháp Compressed Sparse Row (CSR) và tính toán hệ số tương quan cosine
A_sparse = sparse.csr_matrix(matrix_norm)
cosine_similarities = cosine_similarity(A_sparse)
user_cosine_similarity = pd.DataFrame(data=cosine_similarities,index=list(df_matrix[author_username]),columns=list(df_matrix[author_username]))
user_cosine_similarity

,007Waffles,00Yasser,04GreatFlick,0maro0,0w0,108YearsOld,109YearsOld,10sion,121mcv,13Funbags,...,zurnderrick,zutterjp48,zwashington-32756,zwjonas,zwolsa,zzapper-2,zzhhhqing,zzyzx000-124-271558,~AleXa~,~PL~
007Waffles,1.000000,0.000987,-0.049221,0.000000,-0.009605,0.003023,0.029818,0.000000,0.0,0.0,...,0.000000,0.000000,-0.004201,0.005320,0.080823,0.000000,0.000000,0.037530,0.000000,0.000000
00Yasser,0.000987,1.000000,0.000000,0.000000,-0.045324,0.000000,0.000000,-0.014054,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.022146,0.000000,0.040310,0.000000,0.000000,0.000000
04GreatFlick,-0.049221,0.000000,1.000000,0.000000,0.000000,0.000000,0.040089,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,-0.117851,0.000000,0.000000,0.000000,0.000000,0.000000
0maro0,0.000000,0.000000,0.000000,1.000000,0.010355,0.000000,0.000000,0.021899,0.0,0.0,...,0.000000,0.000000,-0.027847,0.000000,0.000000,0.000000,0.014125,0.000000,0.000000,0.000000
0w0,-0.009605,-0.045324,0.000000,0.010355,1.000000,0.020003,-0.005880,0.046623,0.0,0.0,...,0.013974,0.057868,0.009896,0.000000,-0.051428,0.000000,0.020103,0.000000,0.000000,-0.007077
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zzapper-2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.006961
zzhhhqing,0.000000,0.040310,0.000000,0.014125,0.020103,0.000000,0.000000,-0.052181,0.0,0.0,...,0.046529,0.058750,-0.000202,-0.001888,0.000000,0.000000,1.000000,-0.017432,0.016249,-0.000505
zzyzx000-124-271558,0.037530,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.017432,1.000000,-0.041444,0.000000
~AleXa~,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.074784,0.0,0.0,...,-0.089941,0.000000,0.000000,0.046736,0.000000,0.000000,0.016249,-0.041444,1.000000,0.023270


@@ Todo

Áp dụng thuật toán K-NN để tìm các neighborhood cho từng user với K = 100 sắp xếp theo độ tương tự Cosine đã nhắc ở trên. 

Sẽ có xuất hiện các user có độ tương đồng cosine giống nhau nên ta chỉ lấy một user là đủ vì chúng không đem lại lợi ích cho mô hình. 

In [11]:
# Construct Neighborhood
# Chọn ra top 100 users có độ tương tự cao nhất là các neighbors
#threshold = 0.95
K = 100
neighborhood_combine = {}
for user in user_cosine_similarity.columns:
    #neighborhood = list(user_cosine_similarity[user][(user_cosine_similarity[user] > threshold) & (user_cosine_similarity[user] < user_cosine_similarity[user].max())].index)
    neighborhood = list(user_cosine_similarity[user][(user_cosine_similarity[user] < user_cosine_similarity[user].max())].drop_duplicates(keep='first').nlargest(K).index)
    neighborhood_combine[user] = neighborhood

@@Todo

K-RNN chỉ khác với K-NN ở chỗ ta cần xét thêm một điều kiện để quyết định một user có phải nằm trong tập neighborhood của một user bất kỳ hay không. Ta cần check ở cả hai phía của từng user với nhau, nếu cả hai user đều nằm trong tập neighborhood của user còn lại thì nó được xét là một neighborhood chính quy.

In [12]:
# Construct Neighborhood with expanded k-RNN
# Kiểm tra đặc trưng Neighborhood lẫn nhau giữa một user so với các neighbors của chúng
gamma = 1
neighborhood_list = []
neighborhood_combine_1 = neighborhood_combine.copy()
for key,value in neighborhood_combine.items():
    for v in value:
        if key in neighborhood_combine[v]:
            continue
        else:
            neighborhood_combine[key].remove(v)

In [13]:
df_matrix.replace(0,np.nan,inplace=True)

@@Todo

Dưới đây list top 10 phim phù hợp nhất đối với một user bất kỳ theo độ tương tự Cosine

In [15]:
user_id = 'swowen-84128'
service_used_by_user_id = df_matrix[df_matrix[author_username] == user_id].dropna(axis=1, how='all')
similar_services = df_matrix[df_matrix[author_username].isin(neighborhood_combine[user_id])].dropna(axis=1, how='all')
similar_services.drop(service_used_by_user_id.columns[1:],axis=1, inplace=True, errors='ignore')
item_score = {}

# Loop through items
for i in similar_services.columns[1:]:
  # Create a variable to store the score
  total = 0
  # Create a variable to store the number of scores
  count = 0
  # Loop through similar users
  for u in neighborhood_combine[user_id]:
    if math.isnan(similar_services[i][similar_services[author_username] == u].iloc[0]) is False:
    # Score is the sum of user similarity score
      score = user_cosine_similarity[user_id][u] * similar_services[i][similar_services[author_username] == u].values[0]
      total += score
      count +=1

  item_score[i] = total / count
# Convert dictionary to pandas dataframe
item_score = pd.DataFrame(item_score.items(), columns=['service', 'service_score'])
item_score['count'] = list(similar_services.count().values)[1:]
# Sort the services by score
ranked_item_score = item_score.sort_values(by='service_score', ascending=False)

# Select top m services
m = 100
print("Top 10 bộ phim được khuyến khích cho user",user_id)
ranked_item_score[['service','count','service_score']].head(m)

Top 10 bộ phim được khuyến khích cho user swowen-84128


,service,count,service_score
1186,tt11909878,1,3.641281
589,tt0119217,1,3.641281
1457,tt2463208,1,3.641281
762,tt0268978,1,3.641281
900,tt0406759,1,3.641281
...,...,...,...
37,tt0040724,1,1.113716
662,tt0141926,1,1.104944
1633,tt5311542,1,1.097888
1007,tt0490215,2,1.090907


In [16]:
class Collaborative_filtering():
    def __init__(self, K=100,dist_func=cosine_similarity, uuCF=1):
        """
        Khởi tạo CF với các tham số đầu vào:
            data_matrix: ma trận Utility, gồm 3 cột, mỗi cột gồm 3 số liệu: user_id, item_id, rating.
            K: số lượng láng giềng lựa chọn để dự đoán rating.
            uuCF: Nếu sử dụng uuCF thì uuCF = 1 , ngược lại uuCF = 0. Tham số nhận giá trị mặc định là 1.
            dist_f: Hàm khoảng cách, ở đây sử dụng hàm cosine_similarity của klearn.
            limit: Số lượng items gợi ý cho mỗi user. Mặc định bằng 10.
        """
        self.uuCF = uuCF  # user-user (1) or item-item (0) CF
        self.K = K
        self.dist_func = dist_func
    def jaccard_distance(self,data_matrix):
        cols = data_matrix.columns.to_list()[1:]
        res = 1 - pdist(data_matrix[cols], 'jaccard')
        distance = pd.DataFrame(squareform(res), index=list(data_matrix[author_username]), columns= list(data_matrix[author_username]))
        return distance

    def set_landmarks(self,data_matrix,threshold):
        distance = self.jaccard_distance(data_matrix)
        landmarks = popular_users
        distance_landmarks = distance[landmarks]
        return distance_landmarks

    def similarity(self,data_matrix):
        distance_landmarks = self.set_landmarks(data_matrix,5)
        A_sparse = sparse.csr_matrix(distance_landmarks)
        landmarks_similarities = self.dist_func(A_sparse)
        user_landmarks_cosine_similarity = pd.DataFrame(data=landmarks_similarities,index=list(data_matrix[author_username]),columns=list(data_matrix[author_username]))
        self.user_landmarks_cosine_similarity = user_landmarks_cosine_similarity
        return user_landmarks_cosine_similarity

    def build_neighborhood(self,data_matrix):
        user_landmarks_cosine_similarity = self.similarity(data_matrix)
        neighborhood_combine = {}
        for user in user_landmarks_cosine_similarity.columns:
            neighborhood = list(user_landmarks_cosine_similarity[user][(user_landmarks_cosine_similarity[user] < user_landmarks_cosine_similarity[user].max())].drop_duplicates(keep='first').nlargest(self.K).index)
            neighborhood_combine[user] = neighborhood

        # Construct Neighborhood with expanded k-RNN
        # Kiểm tra đặc trưng Neighborhood lẫn nhau giữa một user so với các neighbors của chúng
        for key,value in neighborhood_combine.items():
            for v in value:
                if key in neighborhood_combine[v]:
                    continue
                else:
                    neighborhood_combine[key].remove(v)

        self.neighborhood_combine = neighborhood_combine

    def fit(self,data_matrix):
        self.data_matrix = data_matrix
        self.build_neighborhood(data_matrix)

    def predict_top_services(self,user_id,item_limit,avg=False):
        self.data_matrix.replace(0,np.nan,inplace=True)
        service_used_by_user_id = self.data_matrix[self.data_matrix[author_username] == user_id].dropna(axis=1, how='all')
        # Ma trận chứa neighborhood và các dịch vụ sử dụng tương ứng                                        
        similar_services = self.data_matrix[self.data_matrix[author_username].isin(self.neighborhood_combine[user_id])].dropna(axis=1, how='all')
        similar_services.drop(service_used_by_user_id.columns[1:],axis=1, inplace=True, errors='ignore')
        item_score = {}
        # Loop through items
        for i in similar_services.columns[1:]:
        # Create a variable to store the score
            total = 0
            # Create a variable to store the number of scores
            count = 0
            # Loop through similar users
            for u in self.neighborhood_combine[user_id]:
                if math.isnan(similar_services[i][similar_services[author_username] == u].iloc[0]) is False:
                # Score is the sum of user similarity score
                    score = self.user_landmarks_cosine_similarity[user_id][u] * similar_services[i][similar_services[author_username] == u].values[0]
                    total += score
                    count+= 1
            if (avg):
                item_score[i] = total / count
            else: item_score[i] = total
        # Convert dictionary to pandas dataframe
        item_score = pd.DataFrame(item_score.items(), columns=['service', 'service_score'])
        item_score['count'] = list(similar_services.count().values)[1:]
        # Sort the services by score
        ranked_item_score = item_score.sort_values(by='service_score', ascending=False).reset_index(drop=True)
        return ranked_item_score[['service','count','service_score']].head(item_limit)

In [ ]:
cf.get_recommend_movies("swowen-84128",10,'popular')

In [ ]:
cf.get_recommend_movies("swowen-84128",10,'favorite')

## Triển khai thuật toán

### Đề xuất top N bộ phim cho một người dùng nhất định - Application

In [18]:
def normalization(data_matrix):
    cols = data_matrix.columns[1:].to_list()
    matrix_norm = data_matrix[cols].subtract(data_matrix[cols].mean(axis=1), axis = 'rows')
    matrix_norm.replace(np.nan,0,inplace=True)
    matrix_norm[author_username] = data_matrix[author_username]
    return matrix_norm

In [19]:
neighborhood_combine_dic = {i:j for i,j in neighborhood_combine.items() if j != []}
ser = pd.Series({k: list(v) for k, v in neighborhood_combine_dic.items()}).str.join('|').str.get_dummies()
ser.replace(0,np.nan,inplace=True)
out = dict(ser.stack().iteritems())

neighbor_list = {'user':[],'neighbor':[],'similarity':[]}
for pair in out:
    neighbor_list['user'].append(pair[0])
    neighbor_list['neighbor'].append(pair[1])
    neighbor_list['similarity'].append(user_cosine_similarity[pair[0]][pair[1]])

In [20]:
user_id = 'swowen-84128'

In [21]:
def get_recommend_movies(movie_neighborhood,final_review,user_id,item_limit,mode='popular'):
    check = False
    # Kiểm tra user_id có trong db không, nếu không thì chạy dự đoán. Nếu có thì lấy từ db lên rồi hiển thị
    if check == False:
        review_matrix = final_review.pivot_table(index=author_username, columns=movie_id, values=ratings).reset_index()
        review_matrix_norm = normalization(review_matrix)
        selectedUser_moviesWatched = set(final_review[final_review['author'] == user_id]['imdb_id'].tolist())
        df_selectedUser_neighbors = movie_neighborhood[movie_neighborhood['user'] == user_id].reset_index()
        selectedUser_neighbors = df_selectedUser_neighbors['neighbor'].tolist()
        test_dict = {}
        for user in selectedUser_neighbors:
            similar_movies = final_review[final_review['author'] == user]['imdb_id'].tolist()
            similar_movies = list(set(similar_movies) - set(selectedUser_moviesWatched))
            for movie in similar_movies:
                get_rating = review_matrix_norm[movie][review_matrix_norm['author'] == user].values[0]
                similarity = df_selectedUser_neighbors[df_selectedUser_neighbors['neighbor'] == user]['similarity'].values[0] * get_rating
                test_dict.setdefault(movie,[]).append(similarity)

        if mode == 'popular':
            final_dict = sorted(test_dict, key=lambda x:len(test_dict[x]), reverse=True)[:item_limit]
        elif mode == 'favorite':
            final_dict = sorted(test_dict, key=lambda x:sum(test_dict[x]), reverse=True)[:item_limit]
        # Thực hiện lưu final_dict vào db theo cấu trúc user_id - movie_id
    else:
        # Lấy từ db lên trả về kết quả theo user_id
        print("")
    return final_dict

In [22]:
# Get movie_neighborhood and final_review table from the db
movie_neighborhood = pd.DataFrame(neighbor_list)
final_review = train_df

In [23]:
# Hai mode là popular và favorite
movie_limit = 5
mode = 'popular'
test_dict = get_recommend_movies(movie_neighborhood,final_review,user_id,movie_limit,mode=mode)

In [25]:
test_dict

['tt6723592', 'tt1051906', 'tt0122933', 'tt8623904', 'tt10451914']

### Collaborative filtering

Có hai mods:
- Mod 1: Thiên về độ phổ biến trong tập neighborhood của người dùng được chọn
- Mod 2: Thiên về cảm nhận của các neighbors có độ tương tự cao so với người dùng được chọn

@@Todo

Ta sẽ tạo một class Collaborative Filtering tổng hợp các kiến thức đã được trình bày ở trên.

In [28]:
class Collaborative_filtering():
    def __init__(self, K=100,dist_func=cosine_similarity, uuCF=1):
        """
        Khởi tạo CF với các tham số đầu vào:
            data_matrix: ma trận Utility, gồm 3 cột, mỗi cột gồm 3 số liệu: user_id, item_id, rating.
            K: số lượng láng giềng lựa chọn để dự đoán rating.
            uuCF: Nếu sử dụng uuCF thì uuCF = 1 , ngược lại uuCF = 0. Tham số nhận giá trị mặc định là 1.
            dist_f: Hàm khoảng cách, ở đây sử dụng hàm cosine_similarity của klearn.
            limit: Số lượng items gợi ý cho mỗi user. Mặc định bằng 10.
        """
        self.uuCF = uuCF  # user-user (1) or item-item (0) CF
        self.K = K
        self.dist_func = dist_func

    def convert_matrix(self,train_df):
        data_matrix = train_df.pivot_table(index=author_username, columns=movie_id, values=ratings)
        data_matrix.reset_index(inplace=True)
        self.data_matrix = data_matrix
        return data_matrix

    def normailization(self,train_df):
        data_matrix = self.convert_matrix(train_df)
        cols = data_matrix.columns.to_list()[1:]
        matrix_norm = data_matrix[cols].subtract(data_matrix[cols].mean(axis=1), axis = 'rows')
        matrix_norm.replace(np.nan,0,inplace=True)
        #self.data_matrix = data_matrix
        return matrix_norm

    def similarity(self,data_matrix):
        #distance_landmarks = self.set_landmarks(data_matrix,5)
        matrix_norm = self.normailization(data_matrix)
        A_sparse = sparse.csr_matrix(matrix_norm)
        landmarks_similarities = self.dist_func(A_sparse)
        user_cosine_similarity = pd.DataFrame(data=landmarks_similarities,index=list(self.data_matrix[author_username]),columns=list(self.data_matrix[author_username]))
        self.user_cosine_similarity = user_cosine_similarity
        return user_cosine_similarity

    def build_neighborhood(self,data_matrix):
        user_cosine_similarity = self.similarity(data_matrix)
        neighborhood_combine = {}
        for user in user_cosine_similarity.columns:
            neighborhood = list(user_cosine_similarity[user][(user_cosine_similarity[user] < user_cosine_similarity[user].max())].drop_duplicates(keep='first').nlargest(self.K).index)
            neighborhood_combine[user] = neighborhood

        # Construct Neighborhood with expanded k-RNN
        # Kiểm tra đặc trưng Neighborhood lẫn nhau giữa một user so với các neighbors của chúng
        for key,value in neighborhood_combine.items():
            for v in value:
                if key in neighborhood_combine[v]:
                    continue
                else:
                    neighborhood_combine[key].remove(v)

        self.neighborhood_combine = neighborhood_combine
        self.saveDb_neighborhood(neighborhood_combine)

    def saveDb_neighborhood(self,neighborhood_combine):
        neighborhood_combine_dic = {i:j for i,j in neighborhood_combine.items() if j != []}
        ser = pd.Series({k: list(v) for k, v in neighborhood_combine_dic.items()}).str.join('|').str.get_dummies()
        ser.replace(0,np.nan,inplace=True)
        out = dict(ser.stack().iteritems())

        neighbor_list = {'user':[],'neighbor':[],'similarity':[]}
        for pair in out:
            neighbor_list['user'].append(pair[0])
            neighbor_list['neighbor'].append(pair[1])
            neighbor_list['similarity'].append(user_cosine_similarity[pair[0]][pair[1]])

        # Save movie_neighborhood to the db
        movie_neighborhood = pd.DataFrame(neighbor_list)
        self.movie_neighborhood = movie_neighborhood
        ###
        
    def fit(self,data_matrix):
        self.final_review = data_matrix
        self.build_neighborhood(data_matrix)

    def get_avg_rating(self,user_id):
        sum = 0
        count = 0
        for i in self.data_matrix[self.data_matrix['author'] == user_id].columns[1:]:
            value = self.data_matrix[self.data_matrix['author'] == user_id][i]
            if math.isnan(value) is False:
                sum+=value
                count+=1
        return float(sum / count)

    def get_recommend_movies(self,user_id,item_limit,mode='popular'):
        check = False
        # Kiểm tra user_id có trong db không, nếu không thì chạy dự đoán. Nếu có thì lấy từ db lên rồi hiển thị
        if check == False:
            review_matrix = self.final_review.pivot_table(index=author_username, columns=movie_id, values=ratings).reset_index()
            review_matrix_norm = normalization(review_matrix)
            selectedUser_moviesWatched = set(self.final_review[self.final_review['author'] == user_id]['imdb_id'].tolist())
            df_selectedUser_neighbors = self.movie_neighborhood[self.movie_neighborhood['user'] == user_id].reset_index()
            selectedUser_neighbors = df_selectedUser_neighbors['neighbor'].tolist()
            test_dict = {}
            for user in selectedUser_neighbors:
                similar_movies = self.final_review[self.final_review['author'] == user]['imdb_id'].tolist()
                similar_movies = list(set(similar_movies) - set(selectedUser_moviesWatched))
                for movie in similar_movies:
                    get_rating = review_matrix_norm[movie][review_matrix_norm['author'] == user].values[0]
                    similarity = df_selectedUser_neighbors[df_selectedUser_neighbors['neighbor'] == user]['similarity'].values[0] * get_rating
                    test_dict.setdefault(movie,[]).append(similarity)

            if mode == 'popular':
                final_dict = sorted(test_dict, key=lambda x:len(test_dict[x]), reverse=True)[:item_limit]
            elif mode == 'favorite':
                final_dict = sorted(test_dict, key=lambda x:sum(test_dict[x]), reverse=True)[:item_limit]
            # Thực hiện lưu final_dict vào db theo cấu trúc user_id - movie_id
        else:
            # Lấy từ db lên trả về kết quả theo user_id
            print("")
        return final_dict

In [29]:
cf = Collaborative_filtering()
cf.fit(train_df)

In [30]:
cf.get_recommend_movies("swowen-84128",10,'popular')

['tt1051906',
 'tt8367814',
 'tt6751668',
 'tt2382320',
 'tt6723592',
 'tt8623904',
 'tt6924650',
 'tt1877830',
 'tt10451914',
 'tt6264654']

In [31]:
cf.get_recommend_movies("swowen-84128",10,'favorite')

['tt0181875',
 'tt0068646',
 'tt0107290',
 'tt6751668',
 'tt3783958',
 'tt0756683',
 'tt0085407',
 'tt0109830',
 'tt4154796',
 'tt0166896']

### Evaluation Method

#### 1. Predict Rating

@@Todo

Ta sẽ sử dụng MAE để đánh giá mức độ chính xác của mô hình. Lý do ta chọn MAE để giảm thiểu khả năng tác động của các outliers vì thật sự trong tập dữ liệu thu được sau khi đã tiền xử lý vẫn tồn tại nhiều outliers có các hành vi khác thường nên ta chỉ đánh giá sơ bộ trước trên tập dữ liệu thu được.

In [ ]:
matrix_norm['author'] = df_matrix['author']

In [ ]:
def predict_rating(movie_neighborhood,final_review,user_id,movie_id):
    avg_rating = final_review[final_review['author'] == user_id]['stars'].mean()
    user_watched = final_review[final_review['imdb_id'] == movie_id]['author'].tolist()
    neighbors = movie_neighborhood[movie_neighborhood['neighbor'].isin(user_watched)][movie_neighborhood['user'] == user_id]
    predicted_rating = 0
    sum_similarity = 0.0000001
    for index,user in neighbors.iterrows():
        currentUser_rating = matrix_norm[matrix_norm['author'] == user['neighbor']][movie_id].values[0]
        predicted_rating += currentUser_rating * user['similarity']
        sum_similarity += user['similarity']
    predicted_rating = predicted_rating / sum_similarity
    predicted_rating += avg_rating
    return predicted_rating

In [ ]:
user_id = "swowen-84128"
movie_id = "tt0109830"

predict_rating(movie_neighborhood,final_review,user_id,movie_id)

C:\Users\HP\AppData\Local\Temp\ipykernel_22812\3861643873.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  neighbors = movie_neighborhood[movie_neighborhood['neighbor'].isin(user_watched)][movie_neighborhood['user'] == user_id]


9.105894925541469

In [ ]:
test_df[test_df['author'] == user_id]

,date,imdb_id,author,stars
90182,2020-07-05,tt0109830,swowen-84128,9


In [ ]:
predicted_df = pd.read_csv("testing_results.csv")

In [ ]:
predicted_df[predicted_df['predicted_stars'] < 0]

,date,imdb_id,author,stars,predicted_stars
2305,2020-09-03,tt2531344,juneebuggy,7,-0.118612
2608,2012-08-10,tt1740707,KnightsofNi11,8,-0.323047
2872,2013-01-13,tt1007029,MLDinTN,4,-0.465555
4017,2015-03-29,tt2358925,Buddy-51,7,-0.767545
4031,2014-08-15,tt2318092,Buddy-51,3,-0.767545
...,...,...,...,...,...
93165,2014-01-23,tt0070917,campbell-russell-a,10,-0.096285
93350,2000-04-02,tt0183523,MickeyTo,0,-0.134240
93352,2000-03-26,tt0174856,MickeyTo,0,-0.612255
93405,2003-04-04,tt0029947,JSanicki,0,-3.143795


In [ ]:
train_df[train_df['author'] == "JSanicki"]

,date,imdb_id,author,stars
196903,2003-02-14,tt0046250,JSanicki,0
197929,2003-03-03,tt0053125,JSanicki,0
220931,2003-03-10,tt0161081,JSanicki,0
228714,2003-02-28,tt0299658,JSanicki,10
228880,2003-02-23,tt0249462,JSanicki,0
251251,2003-03-17,tt0048545,JSanicki,0
268570,2003-03-11,tt0042192,JSanicki,0
281381,2003-02-12,tt0050105,JSanicki,0
318644,2003-02-06,tt0048728,JSanicki,0
340055,2003-03-08,tt0228750,JSanicki,0


In [ ]:
predicted_df = predicted_df[predicted_df['predicted_stars'] >= 0]

In [ ]:
predicted_df

,date,imdb_id,author,stars,predicted_stars
0,2023-03-12,tt0167261,ambusched94,8,6.852256
1,2023-03-07,tt0472043,ambusched94,8,8.655460
2,2023-02-19,tt10648342,ambusched94,1,5.941176
3,2023-02-13,tt0765010,ambusched94,5,5.341177
4,2023-02-12,tt0112642,ambusched94,7,5.941176
...,...,...,...,...,...
93478,2019-01-31,tt0093565,anasismael,9,8.090909
93479,2018-11-20,tt6998518,anasismael,8,4.539211
93480,2022-03-08,tt0109045,elisereid-29666,10,9.816506
93481,2021-09-03,tt0081353,elisereid-29666,10,6.014154


In [ ]:
mae = mean_absolute_error(predicted_df['stars'], predicted_df['predicted_stars'])
print("MAE: ",mae)

MAE:  1.6655550762496887


In [ ]:
#test_df.to_csv("testing_results.csv",index=False)

#### 2. Recommendation Efficiency

@@Todo

Recommendation efficiency ở bài toán này sẽ đánh giá khả năng đề xuất của hệ thống trên tập test có thời gian đánh giá mới nhất so với các đánh giá cũ để kiểm tra xem qua khoảng thời gian nhất định biểu hiện hành vi của người dùng như thế nào tập dữ liệu. Cách đánh giá sẽ tập trung vào top N số lượng phim được đề xuất cho một user bất kỳ và ta sẽ kiểm tra xem trong top N có bao nhiêu phim thực sự đã được rating bởi người dùng.

Công thức sẽ sử dụng được gọi là Precision@k như sau:

**P = (# of top k recommendations that are relevant)/(# of items that are recommended)**


<p><img src="https://cdn.discordapp.com/attachments/895583585518620708/1101876809294950430/image.png"></img></p>

In [ ]:
test_users = list(test_df['author'].unique())

In [ ]:
user_limit = 5000
item_limit = 10

total_hit = 0
count = 0
for user in test_users[:user_limit]:
    user_rating = test_df[test_df['author'] == user]
    recommend_movies = get_recommend_movies(movie_neighborhood,final_review,user,item_limit,mode='favorite')
    valid_movies = list(user_rating[user_rating['stars'] >= 7]['imdb_id'])
    invalid_movies = list(user_rating[user_rating['stars'] < 5]['imdb_id'])
    recommend_hit = (len(list(set(recommend_movies).intersection(valid_movies))) - len(list(set(recommend_movies).intersection(invalid_movies)))) / item_limit
    # precisionK
    total_hit+=recommend_hit
    count+=1

print(total_hit/count)

0.01571999999999998


Độ chính xác của tập dữ liệu rơi vào khoảng 1.5%, rất thấp so với một mô hình có thể sử dụng được. Có nhiều lý do cho việc này như ta đã đề cập từ trước do các user có hành vi khác thường khiến mô hình bị sai lệch so với thực tế hay các phim trong tập kiểm thử không so bằng với các phim trong tập đề xuất khiến chúng bị rơi ra khỏi tập đề xuất. 

Sau khi thực hiện, nhóm để ý thấy vị trí được đề xuất trong tập đề xuất cũng rất quan trọng khi độ chính xác của mô hình sẽ được thể hiện trực tiếp nếu các bộ phim trong tập kiểm thử có các vị trí top cao trong tập đề xuất. Qua đó, chúng em tìm hiểu và thực hiện đánh giá theo công thức Average precision (AP):

<p><img src="https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Recommender-Systems-ML-Metrics-vs-Business-Metrics_27.png?resize=480%2C86&ssl=1"></img></p>

<p><img src="https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Recommender-Systems-ML-Metrics-vs-Business-Metrics_6.png?resize=792%2C229&ssl=1"></img></p>

In [ ]:
def avg_precision(recommendation_hit):
    count = 1
    hit = 0
    precision = 0
    for i in recommendation_hit:
        if i == 1:
            hit+=1
            precision+=  hit / count
        count+=1
    if hit == 0 :
        return 0
    else:
        return precision / hit

In [ ]:
user_limit = 5000
item_limit = 10
mean_avg_prec_K = 0
for user in test_users[:user_limit]:
    user_rating = test_df[test_df['author'] == user]
    recommend_movies = get_recommend_movies(movie_neighborhood,final_review,user,10,mode='favorite')
    valid_movies = list(user_rating[user_rating['stars'] >= 7]['imdb_id'])
    recommendation_hit = avg_precision([1 if i in valid_movies else 0 for i in recommend_movies])
    mean_avg_prec_K+= recommendation_hit
    #break
print(mean_avg_prec_K / user_limit)

0.05066791345427054


Độ chính xác sau khi sử dụng AP đã tăng lên 5% chứng tỏ các bộ phim trong tập kiểm thử thực sự có xuất hiện trong tập đề xuất đều đứng ở vị trí rất cao khiến độ chính xác cao hơn so với Precision@K

## Reference

https://neptune.ai/blog/recommender-systems-metrics#:~:text=Average%20reciprocal%20hit%20rank%20(ARHR,of%20the%20first%20correct%20item

https://www.geeksforgeeks.org/cosine-similarity/

https://medium.com/@aliozan_memetoglu/4-collaborative-filtering-and-knn-f997f8993256

https://link.springer.com/article/10.1007/s10479-016-2367-1

https://www.datarevenue.com/en-blog/building-a-production-ready-recommendation-system